In [1]:
using Plots
using Random
using Statistics
using LinearAlgebra

In [2]:
#= simulation hyperparameters
Lx, Ly - 2D size
dt, dx, dy - temporal and two spatial step sizes
=#

Lx, Ly = 60, 60
dt = 0.01
dx = 0.05
dy = 0.05
Nx, Ny = Int64(Lx/dx), Int64(Ly/dy)

(1200, 1200)

In [3]:
# equations and structures

function initialize_simulation_hyperparameters(Lx::Float64,Ly::Float64,dt::Float64,dx::Float64,dy::Float64,Nx::Int64,Ny::Int64, neutrient_field::Matrix{Float64})
    return Dict("Lx"=>Lx, "Ly"=>Ly, "dt"=>dt, "dx"=>dx, "dy"=>dy, "Nx"=>Nx, "Ny"=>Ny, "neutrient_field"=>neutrient_field)
end

#intended to be inside dict of cell types
function initialize_type_hyperparameters(tau_u::Float64, D::Float64, eta_s::Float64, eta::Float64, gamma::Float64, alpha::Float64, beta::Float64, beta_s::Float64, V_max::Float64, L::Float64)
    return Dict("tau_u" => tau_u, "D" => D, "eta_s" => eta_s, "eta" => eta, "gamma" => gamma, "alpha" => alpha, "beta" => beta, "beta_s" => beta_s, "V_max" => V_max, "L" => L)
end

function extract_hyperparameter_vars(cell_hyp)
    for (key, value) in cell_hyp
        eval(Meta.parse("$key = $value"))
    end
    return tau_u, D, eta_s, eta, gamma, alpha, beta, beta_s, V_max, L
end

mutable struct PhaseField
    field::Matrix{Float64}
    field_laplacian::Matrix{Float64}
    square_term::Matrix{Float64}
    h_field::Matrix{Float64}
    volume::Float64
    type::String
    type_hyp::Dict
end


function PhaseField(hpars::Dict, type::String, type_hyp::Dict)
    #=
    types_hpars is dict of cell hyperparameter dicts
    type is type name
    hpars are simulation hyperparameters
    =#
    Nx, Ny = hpars["Nx"], hpars["Ny"]
    field = zeros(Nx, Ny)
    field_laplacian = zeros(Nx, Ny)
    square_term = zeros(Nx, Ny)
    h_field = zeros(Nx, Ny)
    return PhaseField(field, field_laplacian, square_term, h_field, 0., type, type_hyp)
end

h(f) = f*f*(3-2*f)
kdelta(i,j) = i == j ? 1 : 0
volume(f::Matrix{Float64}) = sum(f)

function laplacian(hpars::Dict, f::Matrix{Float64})
    # rows, cols = hpars["Nx"], hpars["Ny"]
    dx, dy = hpars["dx"], hpars["dy"]
    # Rolling operations
    f_ip1 = circshift(f, (-1, 0))  # f(i+1, j)
    f_im1 = circshift(f, (1, 0))   # f(i-1, j)
    f_jp1 = circshift(f, (0, -1))  # f(i, j+1)
    f_jm1 = circshift(f, (0, 1))   # f(i, j-1)
    # Discrete Laplacian computation
    L = (f_ip1 + f_im1 - 2 * f) / dx^2
    L += (f_jp1 + f_jm1 - 2 * f) / dy^2
    return L
end

function calculate_phi(hpars::Dict, fields::Vector{PhaseField}, type::String)
    #=
    calculates phi value for single cell type (summs fields)
    =#
    phi = zeros(size(fields[1].field))
    for field in fields
        if (field.type == type)
            phi .+= field.field
        end
    end
    return phi
end

function sum_h_fields(fields::Vector{PhaseField}, type::String)
    #=
    summs h(PhaseField.field)
    =#
    sum_h = zeros(size(fields[1]))
    for field in fields
        if (field.type == type)
            phi .+= field.h_field
        end
    end
    return sum_h
end

function meshgrid(x::LinRange, y::LinRange)
    return repeat(x', length(y), 1), repeat(y, 1, length(x))
end

# check if working!
function generate_cell!(field::PhaseField, xcenter::Int64, ycenter::Int64, radius::Float64 = 0.5, smooothness::Float64 = 1)
    x = LinRange(0, Lx, Nx)
    y = LinRange(0, Ly, Ny)
    xv, yv = meshgrid(x, y)
    field.field .+= -tanh.(smoothness .* (sqrt.((xv .- xcenter).^2 .+ (yv .- ycenter).^2) .- radius)) / 2 .+ 0.5
end

function take_position(hpars::Dict, field::PhaseField)
    Lx, Ly, Nx, Ny = hpars["Lx"], hpars["Ly"], hpars["Nx"], hpars["Ny"]

    x = LinRange(0, Lx, Nx)
    y = LinRange(0, Ly, Ny)

    X, Y = meshgrid(x, y)

    total_mass = sum(field.field)
    com_x = sum(X .* field.field) / total_mass
    com_y = sum(Y .* field.field) / total_mass

    return com_x, com_y
end

function cell_divide!(hpars::Dict, field::PhaseField, volLim::Float64, smoothness::Float64)
    vol = field.volume
    if volLim > field.type_hyp["V_max"]
        error("volume limit for cell division should be smaller than absolute volume limit for given specie")
    elseif volu < volLim
        return false
    else
        Nx, Ny = hpars["Nx"], hpars["Ny"]
        new_radius = sqrt(vol / pi * 0.5)
        theta = rand() * 2 * pi
        omega_x, omega_y = cos(theta), sin(theta)
        com_x, com_y = take_position(field.field)
        
        field2 = zeros(Nx, Ny)
        x = LinRange(0, Lx, Nx)
        y = LinRange(0, Ly, Ny)
        xv, yv = meshgrid(x, y)
        new_u_1 = (tanh.(smoothness .* (omega_x .* (xv .- com_x) + omega_y .* (yv .- com_y))) .+ 1).*field.field .* 0.5
        new_u_2 = (-tanh.(smoothness .* (omega_x .* (xv .- com_x) + omega_y .* (yv .- com_y))) .+ 1).*field.field .* 0.5

        field.field = new_u_1
        return new_u_2
    end
end

function update_phase_field!(hpars::Dict, u::PhaseField)
    lap = laplacian(hpars, u.field)
    h_f = h.(u.field)
    u.field_laplacian = lap
    u.h_field = h_f
    u.volume = volume(h_f)
    u.square_term = u.field * (1 .- u.field)
end

update_phase_field! (generic function with 1 method)

In [25]:
# defining ODE

function initialize_simulation_hyperparameters(Lx::Float64,Ly::Float64,dt::Float64,dx::Float64,dy::Float64,Nx::Int64,Ny::Int64)
    return Dict("Lx"=>Lx, "Ly"=>Ly, "dt"=>dt, "dx"=>dx, "dy"=>dy, "Nx"=>Nx, "Ny"=>Ny)
end

#intended to be inside dict of cell types
function initialize_type_hyperparameters(tau_u::Float64, D::Float64, eta_s::Float64, eta::Float64, gamma::Float64, alpha::Float64, beta::Float64, beta_s::Float64, V_max::Float64, L::Float64)
    return Dict("tau_u" => tau_u, "D" => D, "eta_s" => eta_s, "eta" => eta, "gamma" => gamma, "alpha" => alpha, "beta" => beta, "beta_s" => beta_s, "V_max" => V_max, "L" => L)
end

function gs(field::Matrix{Float64}, neutrient_field::Matrix{Float64}, eta_s::Float64, hpars::Dict)
    #=
    field - cellular phase field
    neutrient_field - neutrient phase field
    =#
    result = eta_s .* field .* (1 .- field) .* laplacian(hpars, neutrient_field)
    return result
end

function gint(square_term::Matrix{Float64}, h_field::Matrix{Float64}, phi::Matrix{Float64}, h_field_laplacian::Matrix{Float64}, eta::Float64, gamma::Float64, hpars::Dict)
    #=
    field - cellular phase field
    h_field - h.(field)
    phi - combined fields for given specie
    eta - cell parameter
    =#
    result = eta .* square_term .* laplacian(hpars, phi - h_field) .+ gamma .* square_term .* ( h_field_laplacian)
    return result
end

function f_term(h_field::Matrix{Float64}, phi_vector::Vector{Matrix{Float64}}, phi_types::Vector{String}, fields::Vector{PhaseField}, alpha::Float64, V_max::Float64, field_volume::Float64, field_type::String, beta::Float64)
    #=
    calculates interaction term between cell species
    =#
    result = zeros(Float64, size(h_field))
    result .+= alpha * (V_max - field_volume)
    for j in 1:size(phi_vector,1)
        if field_type == phi_types[j]
            result .-= beta .* (phi_vector[j] - h_field)
        else 
            result .-= beta .* phi_vector[j]
        end
    end
    return result
end

function single_step!(result_matrices::Vector{Matrix{Float64}}, hpars::Dict, fields::Vector{PhaseField}, types::Vector{String}, phis::Vector{Matrix{Float64}})
    #=
    later I should consider adding Runge Kutta integration
    fields - vector of cellular phase fields
    types - vector of all cell type names
    phis - initialized before computation vector of specie fields (length of types)
    =#
    if size(phis) != size(types)
        error("wrong size of types vector or phis vector")
    end

    # calculating phi
    for (i, type) in enumerate(types)
        phis[i] = calculate_phi(hpars, fields, type)
    end
    extract_hyperparameter_vars
    for (i, field) in enumerate(fields)
        # extracting parameters
        tau_u, D, eta_s, eta, gamma, alpha, beta, beta_s, V_max, L = extract_hyperparameter_vars(field.type_hyp)
        
        phi_index = findfirst(==(field.type),types)
        h_field_laplacian = laplacian(hpars, field.h_field)
        volume = field.volume
        
        f_term_value =f_term(field.h_field, phis, types, fields, alpha, V_max, field.volume, field.type, beta)
        gs_value = gs(field.field, hpars["neutrient_field"], eta_s, hpars)
        gint_value = gint(field.square_term, field.h_field, phis[phi_index], h_field_laplacian, eta, gamma, hpars)

        result = D .* field.field_laplacian .+ field.square_term .* (field.field .- 0.5 .+ f_term_value) .+ gint_value .+ gs_value
        result_matrices[i] = result
    end
end


single_step! (generic function with 1 method)

In [5]:
# plotting functions

## setting parameters and defining fields

In [6]:
Lx, Ly = 60., 60.
dt = 0.01
dx = 0.05
dy = 0.05
Nx, Ny = Int64(Lx/dx), Int64(Ly/dy)
neutrient_field = zeros(Nx, Ny)
simulation_hyperparameters = initialize_simulation_hyperparameters(Lx, Ly, dt, dx, dy, Nx, Ny, neutrient_field)

Dict{String, Any} with 8 entries:
  "Ly"              => 60.0
  "neutrient_field" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0…
  "Lx"              => 60.0
  "Nx"              => 1200
  "dy"              => 0.05
  "dt"              => 0.01
  "dx"              => 0.05
  "Ny"              => 1200

In [7]:
tau_u = 1.
D = 0.001

eta_s = 0.
eta = 0.
gamma = 0.
alpha = 1.
beta = 1.
beta_s = 0.

V_max = 1.


L = 1.

smoothness = 10.
volLim = 

type1_hyp = initialize_type_hyperparameters(tau_u, D, eta_s, eta, gamma, alpha, beta, beta_s, V_max, L)
types = ["type1"]
field1 = PhaseField(simulation_hyperparameters, "type1", type1_hyp)
generate_cell!(field1, 5, 5, 0.5, smoothness)
fields = [field1]
for i in 1:size(fields,1)
    update_phase_field!(simulation_hyperparameters, fields[i])
end

In [30]:
for tiemstep in 1:100
    #every iteration
    vector_of_results = [zeros(Nx, Ny) for _ in 1:size(fields,1)]
    phis = [zeros(Nx, Ny) for _ in 1:size(types,1)]
    single_step!(vector_of_results, simulation_hyperparameters, fields, types, phis)
end